In [10]:
# Flowchart with SQUARE (rounded-rect) nodes only
# Exports both PNG and PDF. Feel free to edit labels/layout constants.

# v11 (retry) — Smaller font + hard-wrapped labels to keep text inside boxes.
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch, Polygon
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import textwrap

FIGSIZE = (6.69, 8)
FONT = 7
BOX_FACE = "#eeeeee"
BOX_EDGE = "#555555"
GROUP_FACE = "#f8f8f8"
ARROW = "#444444"

def wrap_label(s, width):
    return "\n".join(textwrap.wrap(s, width=width))

def rbox_center(ax, center, wh, text, wrap_width=24):
    cx, cy = center
    w, h = wh
    rect = FancyBboxPatch((cx - w/2, cy - h/2), w, h,
                          boxstyle="round,pad=0.01,rounding_size=0.012",
                          linewidth=1.0, facecolor=BOX_FACE, edgecolor=BOX_EDGE, zorder=3)
    ax.add_patch(rect)
    ax.text(cx, cy, wrap_label(text, wrap_width), ha="center", va="center",
            fontsize=FONT, color="black", zorder=4)
    return rect

def group_container(ax, center, wh, title):
    cx, cy = center
    w, h = wh
    cont = FancyBboxPatch((cx - w/2, cy - h/2), w, h,
                          boxstyle="round,pad=0.005,rounding_size=0.016",
                          linewidth=1.2, facecolor=GROUP_FACE, edgecolor=BOX_EDGE, zorder=1)
    ax.add_patch(cont)
    ax.text(cx, cy + h/2 + 0.02, title, ha="center", va="bottom",
            fontsize=FONT, fontweight="bold", color="black", zorder=4)
    return cont

def diamond(ax, center, wh, text, wrap_width=26):
    cx, cy = center
    w, h = wh
    pts = np.array([[cx, cy + h/2], [cx + w/2, cy], [cx, cy - h/2], [cx - w/2, cy]])
    poly = Polygon(pts, closed=True, facecolor=BOX_FACE, edgecolor=BOX_EDGE, linewidth=1.2, zorder=3)
    ax.add_patch(poly)
    ax.text(cx, cy, wrap_label(text, wrap_width), ha="center", va="center",
            fontsize=FONT, color="black", zorder=4)
    return poly

def arrow(ax, a, b, rad=0.0):
    ar = FancyArrowPatch(a, b, arrowstyle="->", color=ARROW,
                         connectionstyle=f"arc3,rad={rad}", linewidth=1.0, mutation_scale=12, zorder=2)
    ax.add_patch(ar)
    return ar

# Canvas
fig = plt.figure(figsize=FIGSIZE)
ax = plt.gca()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis("off")

ax.text(0.5, 0.95, "Prep Input data for particle tracking", ha="center", va="center",
        fontsize=FONT, fontweight="bold", color="black")

# Group 1
g1_center = (0.5, 0.70)
g1_size   = (0.88, 0.30)
group_container(ax, g1_center, g1_size, "Finding release points")

row_y1 = g1_center[1]
bw, bh = 0.25, 0.07
gap = 0.07
x_mid = 0.50
x1, x2, x3 = x_mid - (bw + gap), x_mid, x_mid + (bw + gap)

b1 = rbox_center(ax, (x1, row_y1), (bw, bh), "1- Intersect stream watershed with shoreline", wrap_width=25)
b2 = rbox_center(ax, (x2, row_y1), (bw, bh), "2- Snap the points to the middle of plume", wrap_width=25)
b3 = rbox_center(ax, (x3, row_y1), (bw, bh), "3- Create 100 particles at each location", wrap_width=25)

arrow(ax, (x1 + bw/2, row_y1), (x2 - bw/2, row_y1))
arrow(ax, (x2 + bw/2, row_y1), (x3 - bw/2, row_y1))

# Group 2
g2_center = (0.5, 0.60)
g2_size   = (0.88, 0.30)
group_container(ax, g2_center, dia_size, "1- Adding N and P")

row_y2 = g2_center[1]
b4 = rbox_center(ax, (x1, row_y2), (bw, bh), "4- Add unique ID to each particle", wrap_width=25)
b5 = rbox_center(ax, (x2, row_y2), (bw, bh), "5- Extract N and P for each stream watershed (SENSEflux)", wrap_width=25)
b6 = rbox_center(ax, (x3, row_y2), (bw, bh), "6- Assign N and P to particle", wrap_width=25)

arrow(ax, (x1 + bw/2, row_y2), (x2 - bw/2, row_y2))
arrow(ax, (x2 + bw/2, row_y2), (x3 - bw/2, row_y2))

arrow(ax, (g1_center[0], g1_center[1] - g1_size[1]/2), (g2_center[0], g2_center[1] + g2_size[1]/2))

# Diamond
dia_center = (0.5, 0.44)
dia_size   = (0.44, 0.12)
diamond(ax, dia_center, dia_size, "Indirect Nutrient load", wrap_width=26)
arrow(ax, (g2_center[0], g2_center[1] - g2_size[1]/2), (dia_center[0], dia_center[1] + dia_size[1]/2))

# Bottom sequence
y_bottom = 0.30
bw2, bh2 = 0.25, 0.08
xL, xC, xR = 0.20, 0.50, 0.80

rbox_center(ax, (xL, y_bottom), (bw2, bh2), "1-Release \nparticles every two months", wrap_width=28)
rbox_center(ax, (xC, y_bottom), (bw2, bh2), "2-Find particles \nfirst arrived to \ncoastal wetlands", wrap_width=28)
rbox_center(ax, (xR, y_bottom), (bw2, bh2), "3-Sum the \ntotal N and P \nfor each coastal wetland", wrap_width=28)

arrow(ax, (dia_center[0], dia_center[1] - dia_size[1]/2), (xL, y_bottom + bh2/2))
arrow(ax, (xL + bw2/2, y_bottom), (xC - bw2/2, y_bottom))
arrow(ax, (xC + bw2/2, y_bottom), (xR - bw2/2, y_bottom))

loop_y = 0.18
arrow(ax, (xR, y_bottom - bh2/2), (xR, loop_y))
arrow(ax, (xR, loop_y), (xL, loop_y))
arrow(ax, (xL, loop_y), (xL, y_bottom - bh2/2))
ax.text(0.50, loop_y - 0.03, "Repeat for 12 months", ha="center", va="center", fontsize=FONT, color="black")

# ---------------------- export ----------------------
png_path = "/home/abolmaal/modelling/FVCOM/Huron/Figures/flow_swimlane_boxes.png"
pdf_path = "/home/abolmaal/modelling/FVCOM/Huron/Figures/flow_swimlane_boxes.pdf"
plt.savefig(png_path, dpi=300, bbox_inches="tight")
with PdfPages(pdf_path) as pdf:
    pdf.savefig(fig, bbox_inches="tight")
plt.close(fig)

png_path, pdf_path


('/home/abolmaal/modelling/FVCOM/Huron/Figures/flow_swimlane_boxes.png',
 '/home/abolmaal/modelling/FVCOM/Huron/Figures/flow_swimlane_boxes.pdf')